In [2]:
import numpy as np
import matplotlib.pyplot as plt
import geone
import geone.covModel as gcm

In [ ]:
# create functions to compute the equivalent permeability of a 2D field
# This is done by computing the arithmetic mean / or the harmonic mean in the x and y directions sequentially until the final value is obtained

def f_3D(a, direction="x", typ="x"):

    """
    Function to operate a mean between two 2D fields from a 3D field

    Parameters
    ----------
    a : 3D numpy array
        The 3D field
    direction : str
        The direction of the mean to compute. Can be "x", "y" or "z"
    type : str
        
    """

    if typ == "y":

        v1 = a[:, ::2, :]
        v2 = a[:, 1::2, :]

    elif typ == "x":

        v1 = a[:, :, ::2]
        v2 = a[:, :, 1::2]

    elif typ == "z":
            
            v1 = a[::2, :, :]
            v2 = a[1::2, :, :]

    if typ == direction:  # series
        res = 1 / np.mean(1 / np.array([v1, v2]), axis=0)
    else:
        res = np.mean((v1, v2), axis=0)
    
    return res

def test_f():

    """
    Test the function f
    """
    a = np.array([[[0.51394334, 0.77316505],
                   [0.87042769, 0.00804695]],
                  [[0.30973593, 0.95760374],
                   [0.51311671, 0.31828442]]])

    a1 = f_3D(a, direction="x", typ="x")
    a2 = f_3D(a1, direction="x", typ="z")
    a3 = f_3D(a2, direction="x", typ="y")
    res = a3[0, 0, 0]
    assert np.allclose(res, 0.3735857397539791)
    
    a1 = f_3D(a, direction="x", typ="y")
    a2 = f_3D(a1, direction="x", typ="z")
    a3 = f_3D(a2, direction="x", typ="x")
    res = a3[0, 0, 0]
    assert np.allclose(res, 0.5323798506575812)

    print("Test passed")


# from numba import jit
# @jit(nopython=True)
def merge(v1, v2, w1, w2, normalize=True):
    
    """
    Function to merge two vectors v1 and v2 with weights w1 and w2
    Is used here to merge two columns of different sizes
    """

    if normalize:
        sum_w = w1 + w2
        w1 = w1 / sum_w
        w2 = w2 / sum_w

    return w1*v1 + w2*v2 - (w1*w2*(v1 - v2)**2)/(w1*v2 + w2*v1)

In [ ]:
def f(x, y):
    return (x**2 - y + 3*x- y**0.5)**2

In [153]:
%%timeit
f(1, 2)

480 ns ± 21.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [157]:
np.random.seed(14)

a = np.random.rand(2, 2, 2)

a1 = f_3D(a, direction="x", typ="x")
a2 = f_3D(a1, direction="x", typ="z")
a3 = f_3D(a2, direction="x", typ="y")
print(a3[0, 0, 0])

a1 = f_3D(a, direction="x", typ="y")
a2 = f_3D(a1, direction="x", typ="z")
a3 = f_3D(a2, direction="x", typ="x")
print(a3[0, 0, 0])

0.3735857397539791
0.5323798506575812


In [143]:
test_f()

Test passed


In [175]:
np.hstack((a, a)).shape

(2, 4, 2)

In [ ]:
def find_c_3D(k_field, direction="x", typ="min"):
    
    if typ == "min":
        if direction == "x":
            l_typ = ["x", "y", "z"]
        elif direction == "y":
            l_typ = ["y", "x", "z"]
        elif direction == "z":
            l_typ = ["z", "x", "y"]
    elif typ == "max":
        if direction == "x":
            l_typ = ["y", "z", "x"]
        elif direction == "y":
            l_typ = ["x", "z", "y"]
        elif direction == "z":
            l_typ = ["x", "y", "z"]
    
    a = np.copy(k_field)
    o = 0
    while a.flatten().shape[0] > 1:
        
        # check if the number of elements is odd
        rm_col = False
        rm_row = False
        rm_lay = False
        sim = True
        if l_typ[o] == "x":
            if a.shape[2] > 1:
                if a.shape[2] % 2 == 1:
                    # remove the last column
                    rm_col = a[:, :, -1]
                    a = a[:, :, :-1]
            else:
                sim = False

        elif l_typ[o] == "y":
            if a.shape[1] > 1:
                if a.shape[1] % 2 == 1:
                    # remove the last row
                    rm_row = a[:, -1, :]
                    a = a[:, :-1, :]
            else:
                sim = False
        elif l_typ[o] == "z":
            if a.shape[0] > 1:
                if a.shape[0] % 2 == 1:
                    # remove the last layer
                    rm_lay = a[-1, :, :]
                    a = a[:-1, :, :]
            else:
                sim = False
        if sim:
            a = f_3D(a, direction=direction, typ=l_typ[o])

        # merge the removed column, row or layer
        if rm_col is not False:
            new_col = merge(rm_col, a[:, :, -1], 1, 2)
            a = np.dstack((a, new_col.reshape(a.shape[0], a.shape[1], 1)))
        elif rm_row is not False:
            new_row = merge(rm_row, a[:, -1, :], 1, 2)
            a = np.hstack((a, new_row.reshape(a.shape[0], 1, a.shape[2])))
        elif rm_lay is not False:
            new_lay = merge(rm_lay, a[-1, :, :], 1, 2)
            a = np.vstack((a, new_lay.reshape(1, a.shape[1], a.shape[2])))

        o += 1
        if o == 3:
            o = 0
    print(a)
    return a[0, 0, 0]

In [218]:
b[:, -1, :]

array([[0.80802655, 0.81537842, 0.00955447],
       [0.72833292, 0.50149933, 0.14661437],
       [0.11131077, 0.49663784, 0.2763581 ]])

In [241]:
find_c_3D(a, direction="x", typ="min")

[[[0.37358574]]]


0.373585739753979

In [325]:
b = np.random.rand(2, 1, 1)
find_c_3D(b, direction="x", typ="max")

IndexError: index -1 is out of bounds for axis 1 with size 0

In [326]:
1 % 2

1